In [7]:
import numpy as np
import pprint
import xlrd
from copy import deepcopy


np.set_printoptions(precision=2) # 设置矩阵输出精度,保留两位小数

'''
def jmax(a):
	"""
	数据标准化
	最大值规格化法
	"""
	a=np.array(a)
	c=np.zeros_like(a,dtype=float)
	for j in range(c.shape[1]): # 遍历c的列
		for i in range(c.shape[0]): # 遍历c的列
			c[i,j]=a[i,j]/np.max(a[:,j])
	return c
'''

def alike(a):
	"""
	模糊相似矩阵
	最大最小法
	"""
	#a=jmax(a) # 用标准化后的数据
	c=np.zeros((a.shape[0],a.shape[0]),dtype=float)
	mmax=[]
	mmin=[]
	for i in range(c.shape[0]): # 遍历c的行
		for j in range(c.shape[0]): # 遍历c的行
			mmax.extend([np.fmax(a[i,:],a[j,:])]) # 取i和和j行的最大值,即求i行和j行的并
			mmin.extend([np.fmin(a[i,:],a[j,:])]) # 取i和和j行的最大值,即求i行和j行的交
	for i in range(len(mmax)):
		mmax[i]=np.sum(mmax[i]) # 求并的和
		mmin[i]=np.sum(mmin[i]) # 求交的和
	mmax=np.array(mmax).reshape(c.shape[0],c.shape[1]) # 变换为与c同型的矩阵
	mmin=np.array(mmin).reshape(c.shape[0],c.shape[1]) # 变换为与c同型的矩阵
	for i in range(c.shape[0]): # 遍历c的行
		for j in range(c.shape[1]): # 遍历c的列
				c[i,j]=mmin[i,j]/mmax[i,j] # 赋值相似度
	return c


def hecheng(a,b):
	"""
	求模糊是矩阵a和模糊矩阵b的合成
	"""
	a,b = np.array(a),np.array(b)
	c=np.zeros_like(a.dot(b))
	for i in range(a.shape[0]): # 遍历a的行元素
		for j in range(b.shape[1]): # 遍历b的列元素
			empty=[]
			for k in range(a.shape[1]):
				empty.append(min(a[i,k],b[k,j])) # 行列元素比小
			c[i,j]=max(empty) # 比小结果取大
	return c


def bibao(a):
	"""
	求模糊矩阵a的闭包
	"""
	a = alike(a) # 用模糊相似矩阵
	c=a
	while True:
		m=c
		c=hecheng(hecheng(a,c),hecheng(a,c))
		if (c==m).all(): # 闭包条件
			return np.around(c,decimals=2) # 返回传递闭包,四舍五入,保留两位小数
			break
		else:
			continue


def julei(a,g):
	a=bibao(a) # 用传递闭包
	c=np.zeros_like(a,dtype=int)
	for i in range(c.shape[0]):
		for j in range(c.shape[1]):
			if a[i,j]>=g:
				c[i,j]=1
			else:
				c[i,j]=0
	return c


def result(a,g):
	"""
	模糊聚类分析结果展示
	"""
	a = julei(a,g)
	c=[] # 同类聚合
	for i in range(len(a)):
		x=[]
		for j in range(i,len(a)):
			if a[i][j]==1:
				x.append(j)
			else:
				continue
		c.append(x)
	d=[] # 删除重复1
	for i in range(len(c)):
		for j in range(i+1,len(c)):
			for k in range(len(c[j])):
				if c[j][k] in c[i]:
					d.append(c[j])
					break
				else:
					continue
	dd=[] # 删除重复2
	for i in range(len(d)):
		for j in range(i+1,len(d)):
			if d[i]==d[j]:
				dd.append(d[j])
	for i in range(len(dd)): # 删除重复3
		try:
			d.pop(d.index(dd[i]))
		except ValueError:
			continue
	for i in range(len(d)): # 删除重复4
		try:
			c.pop(c.index(d[i]))
		except ValueError:
			continue
	for i in range(len(dd)): # 删除重复5
		try:
			c.pop(c.index(dd[i]))
		except ValueError:
			continue
	for i in range(len(c)):
		for j in range(len(c[i])):
			c[i][j]+=1
	return c

def read_excel(path):
	#打开exel
	workbook = xlrd.open_workbook(path,'r') 
	#获得sheet1名字
	sheet1 = workbook.sheets()[0] 
	rows = sheet1.nrows
	cols = sheet1.ncols
	#建立特征指标矩阵
	x_matrix = np.zeros((rows-1,cols-1),dtype = 'float')
	#第一行是列名，从第二行开始
	for i in range(0,rows-1):
		#第一列是序号，从第二列开始
		for j in range(0,cols-1):
			x_matrix[i][j]= sheet1.cell(i+1,j+1).value
	return x_matrix

def main():
	"""
	测试代码
	"""
	path = r'C:\Users\Forry\Desktop\1.xlsx'
	# 0.读取数据
	x = read_excel(path)
	g=0.63

	print("原始数据\n",np.array(x))
	#print("\n数据标准化(最大值规格化法)\n",jmax(x))
	print("\n模糊相似矩阵(最大最小法)\n",alike(x))
	print("\n传递闭包\n",bibao(x))
	print("\n模糊聚类分析矩阵(lambda=%0.2f)\n"%g,julei(x,g))
	print("\n模糊聚类结果(lambda=%0.2f)\n"%g,result(x,g))
	

if __name__ == "__main__":
	main()


原始数据
 [[0.89 0.05 0.   ... 0.2  0.63 0.  ]
 [0.91 0.26 0.6  ... 0.94 0.9  0.95]
 [0.87 0.35 0.2  ... 0.59 0.6  0.52]
 ...
 [0.8  0.74 0.6  ... 0.97 0.93 0.94]
 [0.74 0.84 0.   ... 0.91 0.71 0.91]
 [0.83 0.52 0.8  ... 0.97 0.97 0.98]]

模糊相似矩阵(最大最小法)
 [[1.   0.71 0.77 ... 0.69 0.74 0.72]
 [0.71 1.   0.85 ... 0.89 0.82 0.87]
 [0.77 0.85 1.   ... 0.81 0.82 0.81]
 ...
 [0.69 0.89 0.81 ... 1.   0.86 0.91]
 [0.74 0.82 0.82 ... 0.86 1.   0.88]
 [0.72 0.87 0.81 ... 0.91 0.88 1.  ]]

传递闭包
 [[1.   0.83 0.83 ... 0.83 0.83 0.83]
 [0.83 1.   0.85 ... 0.92 0.9  0.91]
 [0.83 0.85 1.   ... 0.85 0.85 0.85]
 ...
 [0.83 0.92 0.85 ... 1.   0.9  0.91]
 [0.83 0.9  0.85 ... 0.9  1.   0.9 ]
 [0.83 0.91 0.85 ... 0.91 0.9  1.  ]]

模糊聚类分析矩阵(lambda=0.63)
 [[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]

模糊聚类结果(lambda=0.63)
 [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 